In [1]:
import sys
!{sys.executable} -m pip install PyAthena


    100% |████████████████████████████████| 829kB 28.2MB/s ta 0:00:01
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [72]:
from pyathena import connect
import pandas as pd
conn = connect(s3_staging_dir='s3://aws-athena-query-results-984073016564-us-west-2/sagemaker', region_name='us-west-2')
df = pd.read_sql("SELECT distinct * FROM retsdata.union_geo2 WHERE city = 'San Diego'", conn)

In [73]:
len(df)

168786

In [74]:
df.sort_values(by=['zip5'])[:10]

,streetnumber,streetdirprefix,streetname,city,state,zip5,directions,cmplx,neighborhd
161619,2626,None,Coronado Ave,San Diego,CA,88888,"I-5 south, exit on Coronado Ave, left turn Cro...",South Sd,South Sd
8375,2626,None,Coronado Ave,San Diego,CA,88888,"I-5 south, exit on Coronado Ave, left turn",Helena Mobile Home Park,Helena Mobile Park
7005,3033,None,Via Asoleado,San Diego,CA,91901,8E - Tavern Rd (go South) - S Grade (go East) ...,Rancho Palo Verde,Alpine
109040,1915,None,Ranchito Ln,San Diego,CA,91901,None,None,Alpine
5226,2123,None,DELAND CT,San Diego,CA,91901,S Grade to Deland Dr to Deland Ct,None,Alpine
90720,2123,None,DELAND CT,San Diego,CA,91901,S Grade to Deland Dr to Deland Ct Cross Street...,Alpine,Alpine
158224,1915,None,Ranchito Ln,San Diego,CA,91901,Cross Street: Alpine Heights.,Alpine,Alpine
12300,1434,None,Marshall Rd,San Diego,CA,91901,Cross Street: Alpine Blvd.,Alpine,Alpine
58473,3033,None,Via Asoleado,San Diego,CA,91901,8E - Tavern Rd (go South) - S Grade (go East) ...,None,Rancho Palo Verde
94431,5702,None,Yearling Court,San Diego,CA,91902,Cross Street: Corral Canyon Road.,Bonita,Bonita


In [162]:
import tensorflow as tf
import numpy as np

In [84]:
df2 = df['streetnumber'].map(lambda s: '{!s:6.6}'.format(str(s))) + ' ' + df['streetname'].map(lambda s: '{!s:20.20}'.format(s)) + ' ' + df['city'] + ' ' + df['zip5']

In [87]:
df2[:8]

0    6314   Friars Road          San Diego 92108
1    4482   59th Street          San Diego 92115
2    3453   60th St.             San Diego 92115
3    2150   Camino De La Reina   San Diego 92108
4    5474   Caminito Exquisito   San Diego 92130
5    10993  Caminito Arcada      San Diego 92131
6    4540   Wilson Avenue        San Diego 92116
7    3985   Wabaska Drive        San Diego 92107
dtype: object

In [94]:
df3 = df2.map(lambda s: list(map(ord, list(s))))

In [96]:
df3[:8]

0    [54, 51, 49, 52, 32, 32, 32, 70, 114, 105, 97,...
1    [52, 52, 56, 50, 32, 32, 32, 53, 57, 116, 104,...
2    [51, 52, 53, 51, 32, 32, 32, 54, 48, 116, 104,...
3    [50, 49, 53, 48, 32, 32, 32, 67, 97, 109, 105,...
4    [53, 52, 55, 52, 32, 32, 32, 67, 97, 109, 105,...
5    [49, 48, 57, 57, 51, 32, 32, 67, 97, 109, 105,...
6    [52, 53, 52, 48, 32, 32, 32, 87, 105, 108, 115...
7    [51, 57, 56, 53, 32, 32, 32, 87, 97, 98, 97, 1...
dtype: object

In [158]:
N_CODES = 128
def maprow(row):
    sess = tf.Session()
    v = sess.run(tf.one_hot(row, N_CODES))
    return [flatten for sub in v for flatten in sub]
    

In [143]:
train_inputs = df3[:10].map(maprow)
train_inputs

0    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
5    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
6    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
7    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
8    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
9    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
dtype: object

In [160]:
train_labels = train_inputs
len(train_inputs[0])

5504

In [149]:
X = tf.placeholder(tf.float32, shape=[1, len(train_inputs[0])])
Y = tf.placeholder(tf.float32, shape=[1, len(train_labels[0])])

In [150]:
HIDDEN=10
parameters = {
		'W1': tf.Variable(tf.random_normal([len(train_inputs[0]), HIDDEN])),
		'b1': tf.Variable(tf.random_normal([HIDDEN])),
		'W2': tf.Variable(tf.random_normal([HIDDEN, len(train_labels[0])])),
		'b2': tf.Variable(tf.random_normal([len(train_labels[0])]))
}

In [ ]:
num_epochs = 2000
learning_rate = 0.0005

In [154]:
def neural_net(X,parameters):
	Z1 = tf.add(tf.matmul(X, parameters['W1']), parameters['b1'])
	A2 = tf.nn.relu(Z1)
	Z2 = tf.add(tf.matmul(A2, parameters['W2']), parameters['b2'])
	return Z2

In [165]:
def train(X):
    SAMPLES = len(train_inputs)
    INPUT_VAR_CODES = len(train_inputs[0])
    INPUT_VARS = INPUT_VAR_CODES // N_CODES
    OUTPUT_VAR_CODES = len(train_labels[0])
    OUTPUT_VARS = OUTPUT_VAR_CODES // N_CODES
    Z = neural_net(X,parameters)
    costs = []
    optimizers = []
    for i in range(OUTPUT_VARS):
        c = Z[0][N_CODES*i:N_CODES*i+N_CODES]
        costs.append(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=c,  labels=Y[0][N_CODES*i:N_CODES*i+N_CODES])))
        optimizer_k = optimizers.append(tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(costs[i]))
    optimizer = tf.group(*optimizers)
    tf.Print('costs:',costs)
    cost = tf.reduce_sum(costs)
    print(costs)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    with tf.Session() as sess:
        sess.run(init)
        epoch = 0
        while epoch<num_epochs:
            for i in range(SAMPLES):
                _ , c = sess.run([optimizer, cost], feed_dict={
                    X: np.reshape(train_inputs[i],[1,INPUT_VAR_CODES]), 
                    Y: np.reshape(train_labels[i],[1,OUTPUT_VAR_CODES])
                }) 
                if c <= 0.000001:
                    saver.save(sess, '{}\\model.ckpt'.format(dir_path))
                    epoch = num_epochs
                    break
            if epoch % 200 == 0:
                print ("Cost after epoch %i: %f" % (epoch+200, c))
            epoch += 1

    # Test predictions by computing the output using training set as input
    with tf.Session() as sess:
        sess.run(init)
        saver = tf.train.import_meta_graph('model.ckpt.meta')
        saver.restore(sess,'model.ckpt')
        for row in range(len(train_inputs)):

            g = train_inputs[row]
            g = np.reshape(g,[1,INPUT_VAR_CODES])
            output = neural_net(g,parameters)
            outputs = []
            for i in range(OUTPUT_VARS):
                kk = tf.nn.softmax(output[0][N_CODES*i : N_CODES*i+N_CODES])
                outputs.append(kk)
                
            
            out = sess.run(outputs)
            out = np.reshape(list(map(list,out)), [1, OUTPUT_VAR_CODES])[0].tolist()
            out = list(map(lambda x: float("%.1f" % x), out))
            print("\nROW #" + str(row))
            print("Expected: " + str(train_labels[row]))
            row = row + 1
            print("Actual..: " + str(out))

In [ ]:
train(X)

[<tf.Tensor 'Mean_86:0' shape=() dtype=float32>, <tf.Tensor 'Mean_87:0' shape=() dtype=float32>, <tf.Tensor 'Mean_88:0' shape=() dtype=float32>, <tf.Tensor 'Mean_89:0' shape=() dtype=float32>, <tf.Tensor 'Mean_90:0' shape=() dtype=float32>, <tf.Tensor 'Mean_91:0' shape=() dtype=float32>, <tf.Tensor 'Mean_92:0' shape=() dtype=float32>, <tf.Tensor 'Mean_93:0' shape=() dtype=float32>, <tf.Tensor 'Mean_94:0' shape=() dtype=float32>, <tf.Tensor 'Mean_95:0' shape=() dtype=float32>, <tf.Tensor 'Mean_96:0' shape=() dtype=float32>, <tf.Tensor 'Mean_97:0' shape=() dtype=float32>, <tf.Tensor 'Mean_98:0' shape=() dtype=float32>, <tf.Tensor 'Mean_99:0' shape=() dtype=float32>, <tf.Tensor 'Mean_100:0' shape=() dtype=float32>, <tf.Tensor 'Mean_101:0' shape=() dtype=float32>, <tf.Tensor 'Mean_102:0' shape=() dtype=float32>, <tf.Tensor 'Mean_103:0' shape=() dtype=float32>, <tf.Tensor 'Mean_104:0' shape=() dtype=float32>, <tf.Tensor 'Mean_105:0' shape=() dtype=float32>, <tf.Tensor 'Mean_106:0' shape=() 